In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import qiskit
from qiskit import transpile, assemble
from qiskit.visualization import *

In [9]:
# Concentrating on the first 100 samples
n_samples = 100

X_train = datasets.MNIST(root='./data', train=True, download=True,
                         transform=transforms.Compose([transforms.ToTensor()]))

# Leaving only labels 0 and 1 
idx = np.append(np.where(X_train.targets == 0)[0][:n_samples], 
                np.where(X_train.targets == 1)[0][:n_samples])

X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]

train_loader = torch.utils.data.DataLoader(X_train, batch_size=1, shuffle=True)
n_samples = 50

X_test = datasets.MNIST(root='./data', train=False, download=True,
                        transform=transforms.Compose([transforms.ToTensor()]))

idx = np.append(np.where(X_test.targets == 0)[0][:n_samples], 
                np.where(X_test.targets == 1)[0][:n_samples])

X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]

test_loader = torch.utils.data.DataLoader(X_test, batch_size=1, shuffle=True)

In [16]:
# define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__.
# Every nn.Module subclass implements the operations on input data in the forward method.
class Net_c(nn.Module):
    def __init__(self):
        super(Net_c, self).__init__()
        self.conv1=nn.Conv2d(1,6, kernel_size=5)
        self.conv2=nn.Conv2d(6, 16, kernel_size=5)
        self.dropout=nn.Dropout2d()
        self.fc1=nn.Linear(256, 64)
        self.fc2=nn.Linear(64,1)
#         self.hybrid=Hybrid(qiskit.Aer.get_backend('qasm_simulator'), 100, np.pi / 2)
    def forward(self, x):
        x=F.relu(self.conv1(x))
        x=F.max_pool2d(x, 2)
        x=F.relu(self.conv2(x))
        x=F.max_pool2d(x,2)
        x=self.dropout(x)
        x=x.view(1,-1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
#         x=self.hybrid(x)
        return torch.cat((x,1-x),-1)
# To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!
# create an instance of Net_c
model_c = Net_c()
loss_func = nn.CrossEntropyLoss()
# nn.MSELoss (Mean Square Error) for regression tasks, and nn.NLLLoss (Negative Log Likelihood)
# initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.
optimizer = optim.Adam(model_c.parameters(), lr=0.001)

epochs = 20
loss_list = []

model_c.train()
for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
#       reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
        optimizer.zero_grad()
        # Forward pass
        output = model_c(data)
        # Calculating loss
        loss = loss_func(output, target)
        # Backpropagate the prediction loss (deposits the gradients of the loss w.r.t. each parameter.)
        loss.backward()
        # Optimize the weights (adjust the parameters by the gradients collected in the backward pass.)
        optimizer.step()
        
        total_loss.append(loss.item())
    loss_list.append(sum(total_loss)/len(total_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(
        100. * (epoch + 1) / epochs, loss_list[-1]))

Training [5%]	Loss: 0.2367
Training [10%]	Loss: 0.0145
Training [15%]	Loss: 0.0442
Training [20%]	Loss: 0.0047
Training [25%]	Loss: 0.0023
Training [30%]	Loss: 0.0015
Training [35%]	Loss: 0.0053
Training [40%]	Loss: 0.0007
Training [45%]	Loss: 0.0008
Training [50%]	Loss: 0.0001
Training [55%]	Loss: 0.0249
Training [60%]	Loss: 0.0050
Training [65%]	Loss: 0.0004
Training [70%]	Loss: 0.0008
Training [75%]	Loss: 0.0151
Training [80%]	Loss: 0.0050
Training [85%]	Loss: 0.0021
Training [90%]	Loss: 0.0004
Training [95%]	Loss: 0.0002
Training [100%]	Loss: 0.0001


In [17]:
model_c.eval()
with torch.no_grad():
    
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model_c(data)
        
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        loss = loss_func(output, target)
        total_loss.append(loss.item())
        
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_loader) * 100)
        )

Performance on test data:
	Loss: 0.0001
	Accuracy: 100.0%


In [11]:
# implement a hidden layer for our neural network using a parameterized quantum circuit
# rotation angles for each gate are specified by the components of a classical input vector.
class QuantumCircuit:
    def __init__(self, n_qubits, backend, shots):
        self._circuit=qiskit.QuantumCircuit(n_qubits)
        all_qubits=[i for i in range(n_qubits)]
        self.theta=qiskit.circuit.Parameter('theta')
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        self._circuit.measure_all()
        self.backend=backend
        self.shots=shots
    def run(self, thetas):
        t_qc=transpile(self._circuit,self.backend)
        qobj=assemble(t_qc, shots=self.shots,parameter_binds=[{self.theta:theta} for theta in thetas])
        job=self.backend.run(qobj)
        result=job.result().get_counts(self._circuit)
        counts=np.array(list(result.values()))
        states=np.array(list(result.keys())).astype(float)
        probabilities=counts/self.shots
        expectation=np.sum(states*probabilities)
        return np.array([expectation])
simulator = qiskit.Aer.get_backend('qasm_simulator')

circuit = QuantumCircuit(1, simulator, 100)
print('Expected value for rotation pi {}'.format(circuit.run([np.pi])[0]))
circuit._circuit.draw()

Expected value for rotation pi 0.53


┌───┐ ░ ┌───────────┐ ░ ┌─┐
   q_0: ┤ H ├─░─┤ RY(theta) ├─░─┤M├
        └───┘ ░ └───────────┘ ░ └╥┘
meas: 1/═════════════════════════╩═
                                 0

In [13]:
class HybridFunction(Function):
    @staticmethod
    def forward(ctx, input,quantum_circuit, shift):
        ctx.shift=shift
        ctx.quantum_circuit=quantum_circuit
        expectation_z=ctx.quantum_circuit.run(input[0].tolist())
        result=torch.tensor([expectation_z])
        ctx.save_for_backward(input, result)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        input, expectation_z=ctx.saved_tensors
        input_list=np.array(input.tolist())
        shift_right=input_list+np.ones(input_list.shape)*ctx.shift
        shift_left=input_list-np.ones(input_list.shape)*ctx.shift
        gradients=[]
        for i in range(len(input_list)):
            expectation_right=ctx.quantum_circuit.run(shift_right[i])
            expectation_left=ctx.quantum_circuit.run(shift_left[i])
            gradient=torch.tensor([expectation_right])-torch.tensor([expectation_left])
            gradients.append(gradient)
        gradients=np.array([gradients]).T
        return torch.tensor([gradients]).float()*grad_output.float(), None, None
class Hybrid(nn.Module):
    def __init__(self, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.quantum_circuit=QuantumCircuit(1, backend, shots)
        self.shift=shift
    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit, self.shift)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Conv2d(1,6, kernel_size=5)
        self.conv2=nn.Conv2d(6, 16, kernel_size=5)
        self.dropout=nn.Dropout2d()
        self.fc1=nn.Linear(256, 64)
        self.fc2=nn.Linear(64,1)
        self.hybrid=Hybrid(qiskit.Aer.get_backend('qasm_simulator'), 100, np.pi / 2)
    def forward(self, x):
        x=F.relu(self.conv1(x))
        x=F.max_pool2d(x, 2)
        x=F.relu(self.conv2(x))
        x=F.max_pool2d(x,2)
        x=self.dropout(x)
        x=x.view(1,-1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        x=self.hybrid(x)
        return torch.cat((x,1-x),-1)
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.NLLLoss()

epochs = 20
loss_list = []

model.train()
for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        # Calculating loss
        loss = loss_func(output, target)
        # Backward pass
        loss.backward()
        # Optimize the weights
        optimizer.step()
        
        total_loss.append(loss.item())
    loss_list.append(sum(total_loss)/len(total_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(
        100. * (epoch + 1) / epochs, loss_list[-1]))

Training [5%]	Loss: -0.8237
Training [10%]	Loss: -0.9231
Training [15%]	Loss: -0.9437
Training [20%]	Loss: -0.9469
Training [25%]	Loss: -0.9453
Training [30%]	Loss: -0.9557
Training [35%]	Loss: -0.9613
Training [40%]	Loss: -0.9729
Training [45%]	Loss: -0.9720
Training [50%]	Loss: -0.9786
Training [55%]	Loss: -0.9738
Training [60%]	Loss: -0.9787
Training [65%]	Loss: -0.9820
Training [70%]	Loss: -0.9794
Training [75%]	Loss: -0.9866
Training [80%]	Loss: -0.9877
Training [85%]	Loss: -0.9893
Training [90%]	Loss: -0.9897
Training [95%]	Loss: -0.9919
Training [100%]	Loss: -0.9929


In [14]:
model.eval()
with torch.no_grad():
    
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data)
        
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        loss = loss_func(output, target)
        total_loss.append(loss.item())
        
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_loader) * 100)
        )

Performance on test data:
	Loss: -0.9870
	Accuracy: 100.0%


In [15]:
idx.shape

(100,)